In [103]:
import pandas as pd
import numpy as np
train = pd.read_csv("train_data1.csv", encoding='cp949')
test = pd.read_csv('test_data.csv', encoding='cp949')

시간당 내부 평균온도: 4도~40도
시간당 내부 평균습도: 0% ~ 100%
시간당 내부 평균 co2 농도 : 0ppm ~ 1200 ppm
시간당 평균 EC : 0 ~ 8
시간당 분무량 : 0 ~ 3000 / 일간 누적 분무량 0 ~ 72,000
시간당 백색광량 : 0 ~ 120,000 / 일간 누적 백색광량 0 ~ 2,880,000
시간당 적색광량 : 0 ~ 120,000 / 일간 누적 적색광량 0 ~ 2,880,000 
시간당 청색광량 : 0 ~ 120,000 / 일간 누적 청색광량 0 ~ 2,880,000
시간당 총광량 : 0 ~ 120,000 / 일간 누적 총광량 0 ~ 2,880,000

In [104]:
def out_value(df):
    df.loc[(df['내부온도관측치'] < 4) | (df['내부온도관측치'] > 40), '내부온도관측치'] = np.nan
    df.loc[(df['내부습도관측치'] < 0) | (df['내부습도관측치'] > 100), '내부습도관측치'] = np.nan
    df.loc[(df['co2관측치'] < 0) | (df['co2관측치'] > 1200), 'co2관측치'] = np.nan
    df.loc[(df['ec관측치'] < 0) | (df['ec관측치'] > 8), 'ec관측치'] = np.nan
    df.loc[(df['시간당분무량'] < 0) | (df['시간당분무량'] > 3000), '시간당분무량'] = np.nan
    df.loc[(df['일간누적분무량'] < 0) | (df['일간누적분무량'] > 72000), '일간누적분무량'] = np.nan
    df.loc[(df['시간당백색광량'] < 0) | (df['시간당백색광량'] > 120000), '시간당백색광량'] = np.nan
    df.loc[(df['일간누적백색광량'] < 0) | (df['일간누적백색광량'] > 2880000), '일간누적백색광량'] = np.nan
    df.loc[(df['시간당적색광량'] < 0) | (df['시간당적색광량'] > 120000), '시간당적색광량'] = np.nan
    df.loc[(df['일간누적적색광량'] < 0) | (df['일간누적적색광량'] > 2880000), '일간누적적색광량'] = np.nan
    df.loc[(df['시간당청색광량'] < 0) | (df['시간당청색광량'] > 120000), '시간당청색광량'] = np.nan
    df.loc[(df['일간누적청색광량'] < 0) | (df['일간누적청색광량'] > 2880000), '일간누적청색광량'] = np.nan
    df.loc[(df['시간당총광량'] < 0) | (df['시간당총광량'] > 120000), '시간당총광량'] = np.nan
    df.loc[(df['일간누적총광량'] < 0) | (df['일간누적총광량'] > 2880000), '일간누적총광량'] = np.nan
    return df

In [105]:
train = out_value(train)
test = out_value(test)
train = train.fillna(method='ffill')
test = test.fillna(method='ffill')

In [106]:
train['obs_time'] = [i[:2] for i in train['obs_time']]
test['obs_time'] = [i[:2] for i in test['obs_time']]

In [107]:
x_train = train.drop(['predicted_weight_g', 'Case'], axis=1)
y_train = train['predicted_weight_g']

x_test = test.drop(['predicted_weight_g', 'Case'], axis=1)

In [110]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(x_train, y_train)

LinearRegression()

In [112]:
y_pred = model.predict(x_train)

In [122]:
from sklearn.metrics import mean_squared_error

RMSE = mean_squared_error(y_train, y_pred)**0.5

In [121]:
y_pred = y_pred+10

In [120]:
y_train

0         0.167719
1         0.167719
2         0.167719
3         0.167719
4         0.167719
           ...    
18811    86.591508
18812    86.591508
18813    86.591508
18814    86.591508
18815    86.591508
Name: predicted_weight_g, Length: 18816, dtype: float64

In [123]:
RMSE

26.87071924197845